In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pickle
import os
import json
import numpy as np


Firebase signin

In [3]:

cred = credentials.Certificate("../codeinterface-85b5e-firebase-adminsdk-11q7e-837ba92a03.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [21]:
docs = db.collection('tasks').get()
# save docs in a pickle file called old_tasks.pkl
tasks = {}
for doc in docs:
    doc_dict = doc.to_dict()
    doc_dict['id'] = doc.id 
    tasks[doc.id] = doc_dict


In [22]:
today_date_string = str(np.datetime64('today', 'D'))
with open('old_tasks'+today_date_string+'.pkl', 'wb') as f:
    pickle.dump(tasks, f)

In [23]:
# remove all the documents from the collection
for doc in docs:
    doc.reference.delete()

In [24]:

folder_path = './tasks'  # Replace with the actual folder path

# Initialize a list to store all tasks
all_tasks = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            # Load JSON content
            task_data = json.load(file)
            # Add the task to the list
            all_tasks.append(task_data)

all_tasks = np.array(all_tasks)

In [8]:
len(all_tasks)

17

In [18]:
# find task in all_tasks with field name  is Average
def get_task_index(task_name):
    for i in range(len(all_tasks)):
        if all_tasks[i]['name'] == task_name:
            return i
    # raise error if task not found
    raise ValueError('Task not found')
tutorial_task_index = get_task_index('sum_product')
tasks_chosen = np.array([get_task_index('table_transform_named'), get_task_index('tokenizer')])




In [19]:
all_tasks[tasks_chosen]

array([{'name': 'table_transform_named', 'task_description': '\n+----+-------+---------+---------------------+----------+\n|    |   age | color   | dates               |   height |\n|----+-------+---------+---------------------+----------|\n|  0 |     1 | blue    | 2019-03-06 00:00:00 |  2.72656 |\n|  1 |     4 | blue    | 2019-03-05 00:00:00 |  4.77665 |\n|  2 |     4 | green   | 2019-03-10 00:00:00 |  8.12169 |\n|  3 |    10 | brown   | 2019-03-07 00:00:00 |  4.79977 |\n|  4 |    20 | green   | 2019-03-01 00:00:00 |  3.92785 |\n+----+-------+---------+---------------------+----------+\n\n\nThis is a pandas dataframe provided to you above as input.\n\nYou need to transform it exactly to the following output dataframe:\n\n+----+----------+--------+---------+---------+---------+-------+----------+\n|    | age      |   blue |   brown |   green |   month |   day |   height |\n|----+----------+--------+---------+---------+---------+-------+----------|\n|  0 | Under 18 |      1 |       0 | 

In [25]:

# Extracting data for the tutorial task
tutorial_task = all_tasks[tutorial_task_index]

selected_tasks = [all_tasks[i] for i in tasks_chosen]

# Creating the dictionary with the specified structure
fb_taskgpt = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "gpt35",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilot2',  # Placeholder, as the actual condition is not specified
    'max_tokens': 64,  # Placeholder, adjust as necessary
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_gpt35",
    'task_completed': 0,
}

fb_taskllama = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "togethercomputer/CodeLlama-7b",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilot2',  
    'max_tokens': 64,  
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_llama7",
    'task_completed': 0,

}

fb_tasknomodel = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "togethercomputer/CodeLlama-7b",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilot2',  
    'max_tokens': 64,  
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_nomodel",
    'task_completed': 0,
}

for i in range(10):
    doc_ref = db.collection(u'tasks').document(u'pilot_gpt35'+str(i))
    doc_ref.set(fb_taskgpt)

    doc_ref = db.collection(u'tasks').document(u'pilot_llama7'+str(i))
    doc_ref.set(fb_taskllama)

    doc_ref = db.collection(u'tasks').document(u'pilot_nomodel'+str(i))
    doc_ref.set(fb_tasknomodel)

